## Analysis of Landing pad integration

### Load libraries and functions

In [ ]:
import fcsparser as fcs
import os
import pandas as pd
import warnings
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import math
sns.set(style ="whitegrid")
sns.set_context("poster")
warnings.filterwarnings("once")

### Load files and metadata

In [ ]:
#folder with fcs files in subfolders
folder="C:\\FCS_experiment\\"

ffolderlist = [f for f in os.listdir(folder) if os.path.isdir(os.path.join(folder, f))]


df=pd.DataFrame()
aa=1

#load all fcs files
for fdn in ffolderlist:
    ffilelist = os.listdir(folder+fdn)
    print (fdn)
    plate = fdn.split("_")[2]
    run = fdn.split("_")[3]
    for fn in ffilelist:
        filename, file_ext = os.path.splitext(fn)
        if file_ext ==".fcs":
            path = folder + fdn + "//" + fn
            meta, df1 = fcs.parse(path, meta_data_only=False, reformat_meta=True)
            df1["WellName"]=fn.split("_")[3].split(".")[0]
            df1["WellNumber"]= aa
            df1["Plate"] =plate
            df1["Run"] = run
            df=df.append(df1)
            aa=aa+1

# metadata file
meta = "C:\\metadata.csv"
df1=pd.read_csv(meta, names=["WellName", "Name", "ExperimentName", "Info", "Plate"], dtype= object)

dfnew=df1.merge(df, on=["WellName", "Plate"])


### remove zeros, gate live cells and log data

In [ ]:
#remove zeros
dfnew= dfnew[dfnew > 0]

#remove doublets
dfnew["doublet"] = dfnew["FSC-A"] / dfnew["FSC-H"]
df2=dfnew[(dfnew.doublet < 2)& (dfnew.doublet > 1.25)]
df2=dfnew

#remove small events
df2 =df2[df2["FSC-A"]>0E5]
df2 =df2[(df2["SSC-A"]<2E5)]

#show data for first well
g = sns.jointplot("FSC-A", "SSC-A", data=df2[df2["WellNumber"]==1], s=1)#.plot_joint(sns.kdeplot, zorder=0, n_levels=6);
plt.show()

#log data
for col in df2.columns:
    if df2[col].dtype=="float32":
        df2["log"+ col]=df2[col].apply(math.log10)


### plot distributions

In [ ]:
df23=df2[~np.isnan(df2["logFITC-A"])]

g = sns.FacetGrid(data=df23, row = "Info")
g=g.map(sns.distplot, "logFITC-A", color='g').set(xlim=(0, 6))
plt.savefig(folder + "LP_GDist.eps", type = "eps")
plt.savefig(folder + "LP_GDist.png", type = "png", dpi =300)

g = sns.FacetGrid(data=df23, row = "Info")
g=g.map(sns.distplot, "logBV 421-A", color='b').set(xlim=(1, 4))
plt.savefig(folder + "LP_BDist.eps", type = "eps")
plt.savefig(folder + "LP_BDist.png", type = "png", dpi =300)

g = sns.FacetGrid(data=df23, row = "Info")
g=g.map(sns.distplot, "logPE-CF594-A", color='r').set(xlim=(0, 6))

plt.savefig(folder + "LP_RDist.eps", type = "eps")
plt.savefig(folder + "LP_RDist.png", type = "png", dpi =300)
plt.show()
